In [1]:
#-------------------------------------------------------------------------------------JUPYTER NOTEBOOK SETTINGS-------------------------------------------------------------------------------------
from IPython.core.display import display, HTML                                    
display(HTML("<style>.container { width:100% !important; }</style>"))  

/var/folders/ry/454yhlzx6hd15j7rjv4th0lw0000gn/T/ipykernel_36848/2828031236.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [ ]:
import os
import gc
import re
import librosa
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
from joblib import dump, load

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.manifold import TSNE
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import MobileNetV3Small

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

In [ ]:
x_train, y_train = load('saved_data/train_data.joblib')
x_val, y_val = load('saved_data/val_data.joblib')
x_test, y_test = load('saved_data/test_data.joblib')

In [ ]:
# Ensure all input data is float32
x_train = np.array(x_train, dtype=np.float32)
x_val = np.array(x_val, dtype=np.float32)
x_test = np.array(x_test, dtype=np.float32)

In [ ]:
print("Train data:", x_train.shape, x_train.dtype)
print("Validation data:", x_val.shape, x_val.dtype)
print("Test data:", x_test.shape, x_test.dtype)

# Check for any NaN or inf values in your dataset
print("NaNs in train:", np.isnan(x_train).any())
print("NaNs in validation:", np.isnan(x_val).any())
print("NaNs in test:", np.isnan(x_test).any())

print("Infs in train:", np.isinf(x_train).any())
print("Infs in validation:", np.isinf(x_val).any())
print("Infs in test:", np.isinf(x_test).any())

In [ ]:
# ONEHOT ENCODING THE LABELS
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train_encoded)
y_val_onehot = to_categorical(y_val_encoded)
y_test_onehot = to_categorical(y_test_encoded)

In [ ]:
# Define paths and initial settings
directory = 'saved_data/'
if not os.path.exists(directory):
    os.makedirs(directory)

# Load MobileNetV3 Small pre-trained on ImageNet
base_model = MobileNetV3Small(weights='imagenet', include_top=False, input_shape=(96, 96, 3))

# Freeze the convolutional base
base_model.trainable = False

# Model architecture adapted for audio classification
model = Sequential([
    Reshape((13, 332, 1), input_shape=(13, 332)),  # Reshape input to have 1 channel
    tf.keras.layers.UpSampling2D(size=(7, 1)),  # Upsample to get closer to the expected input size of MobileNetV3
    tf.keras.layers.Conv2D(3, (3, 3), activation='relu', padding='same'),  # Convert to 3 channels
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(y_train_onehot.shape[1], activation='softmax')
])

# Create an Adam optimizer with a specified initial learning rate
optimizer = Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Setup callbacks
early_stopping_monitor = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=1)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.000001, verbose=1)
weights_saver = ModelCheckpoint(f'{directory}mobilenetv3small_finetuned_model_weights_epoch_{epoch}.h5', save_freq=50, save_weights_only=True, verbose=1)

# Train the model
history = model.fit(
    x_train,
    y_train_onehot,
    epochs=300,
    batch_size=256,
    validation_data=(x_val, y_val_onehot),
    callbacks=[weights_saver, early_stopping_monitor, reduce_lr_on_plateau],
    verbose=1
)

# Save final model
model.save(f'{directory}mobilenetv3small_finetuned_model.keras', save_format='tf')

# Save the final training history
from joblib import dump
dump(history.history, f'{directory}mobilenetv3small_finetuned_training_history.joblib')